In [1]:
import pandas as pd
import numpy as np
import naive_bayes as nb

## Иницилизация языковой библиотеки для работы с корнем

In [2]:
try:
    from nltk.stem.snowball import SnowballStemmer
    def get_stem(word,lang=None):
        if lang is None:
            lang = 'en'
        if lang=='ru':
            stemmer = SnowballStemmer('russian')
        else:
            stemmer = SnowballStemmer('english')
        return stemmer.stem(word)
except ModuleNotFoundError:
    def get_stem(word,lang=None):
        if lang is None:
            lang = 'en'
        return word
        pass

# Байесовский классификатор Гаусса

## Игра в гольф. Играть или не играть?

In [3]:
nb_nd=nb.NaiveBayesGaussian()

data_golf_humidity = {'yes': [86, 96, 80, 65, 70, 80, 70, 90, 75], 'no': [85, 90, 70, 95, 91]}
# play golf
print('```Когда будем играть в гольф?```')
print(f'Будем играть, когда влажность: {data_golf_humidity["yes"]} ')
print(f'Не будем играть, когда влажность: {data_golf_humidity["no"]} ')
print()


```Когда будем играть в гольф?```
Будем играть, когда влажность: [86, 96, 80, 65, 70, 80, 70, 90, 75] 
Не будем играть, когда влажность: [85, 90, 70, 95, 91] 



In [4]:
humidity = [74,80,81,82,83]
threshold_value = 0.5
print(f'Пороговое значение: {threshold_value}')
print()

for i in humidity:
    print(f'Будем играть при влажности={i}? ',end='')
    pr = nb_nd.probability(i, main_data=data_golf_humidity['yes'], 
                           opponent_data=data_golf_humidity['no'])
    
    if pr > threshold_value:
        print(f'Yes!',end='')
    else:
        print(f'No!',end='')
    print(f' (p: {pr})')
    print()


Пороговое значение: 0.5

Будем играть при влажности=74? Yes! (p: 0.65)

Будем играть при влажности=80? Yes! (p: 0.54)

Будем играть при влажности=81? Yes! (p: 0.52)

Будем играть при влажности=82? No! (p: 0.5)

Будем играть при влажности=83? No! (p: 0.48)



# Мультиномиальный Байесовский классификатор (для текстов)

## Коты/Собаки
Проверим алгоритм на задаче из лекции, про котов и собак

### Загрузка и подготовка данных

In [5]:
cats_dogs = nb.NaiveBayesMultinomial()

cats_dogs.stem_func = get_stem
cats_dogs.stem_enable = True
cats_dogs.stem_lang = 'ru'
cats_dogs.set_title('Коты', 'Собаки')

train_text = [
    'Кот бежит к будке и говорит мяу.',
    'Белого кота и чёрного кота несут в котоноске.',
    'Большой кот и маленький кот поймали мышь.',
    
    'Собака из будки смотрит на кота.',
    'Собака залезла не в свою будку, а в чужую будку.',
]
train_class = [1, 1, 1, 0, 0]

test_text = 'Белый кот, чёрный кот и рыжий кот идут мимо будки собаки'

df = pd.DataFrame(list(zip( train_class,train_text)),columns=['class1', 'text'])
df_test = pd.DataFrame(list(zip([1], [test_text])),columns=['class1', 'text'])



In [6]:
cats_dogs.load_stop_words('input/stopwords_ru.txt')

count_all, count_cats, count_dogs = cats_dogs.calculate_counters(df=df, column='class1')

print(f'Данные для расчета: {cats_dogs.class_title[cats_dogs.index_class1]}: {count_cats}, {cats_dogs.class_title[cats_dogs.index_class2]}: {count_dogs}')
print('Данные для проверки: {0}: {1}, {2}: {3}'.format(
    cats_dogs.class_title[cats_dogs.index_class1],
    df_test['class1'].loc[df_test['class1'] == 1].count(),
    cats_dogs.class_title[cats_dogs.index_class2],
    df_test['class1'].loc[df_test['class1'] == 0].count()
))



Данные для расчета: Коты: 3, Собаки: 2
Данные для проверки: Коты: 1, Собаки: 0


### Подсчет слов

In [7]:
cats_dogs.train(df=df)
print('Всего слов: {0}, без стоп-слов: {1}'.format(len(cats_dogs.words_all), len(cats_dogs.words_without_stop)))

Всего слов: 26, без стоп-слов: 19


In [8]:
print(cats_dogs.words_without_stop)

defaultdict(<function NaiveBayesMultinomial.train.<locals>.<lambda> at 0x7f14926d3e20>, {'кот': [5, 1], 'беж': [1, 0], 'будк': [1, 3], 'говор': [1, 0], 'мя': [1, 0], 'бел': [1, 0], 'ч': [1, 0], 'рног': [1, 0], 'несут': [1, 0], 'котоноск': [1, 0], 'больш': [1, 0], 'маленьк': [1, 0], 'пойма': [1, 0], 'мыш': [1, 0], 'собак': [0, 2], 'смотр': [0, 1], 'залезл': [0, 1], 'сво': [0, 1], 'чуж': [0, 1]})


### Проверка на тестовой выборке

In [9]:
print('Текст "{0}"'.format(df_test.iloc[0]['text']))

Текст "Белый кот, чёрный кот и рыжий кот идут мимо будки собаки"


#### с учетом стоп слов

In [10]:
print('### С учетом стоп слов ###')
print('входит в группу "{0}" с вероятностью {1}'.format(
    cats_dogs.class_title[cats_dogs.index_class1],
    cats_dogs.probability(text=df_test.iloc[0]['text'], mode=cats_dogs.index_class1,
                          words=cats_dogs.words_without_stop)
))
print('входит в группу "{0}" с вероятностью {1}'.format(
    cats_dogs.class_title[cats_dogs.index_class2],
    cats_dogs.probability(text=df_test.iloc[0]['text'], mode=cats_dogs.index_class2,
                          words=cats_dogs.words_without_stop)
))

### С учетом стоп слов ###
входит в группу "Коты" с вероятностью 0.95
входит в группу "Собаки" с вероятностью 0.05


#### без учета стоп слов

In [11]:
print('### Без учета стоп слов ###')
print('входит в группу "{0}" с вероятностью {1}'.format(
    cats_dogs.class_title[cats_dogs.index_class1],
    cats_dogs.probability(text=df_test.iloc[0]['text'], mode=cats_dogs.index_class1,
                          words=cats_dogs.words_all)
))
print('входит в группу "{0}" с вероятностью {1}'.format(
    cats_dogs.class_title[cats_dogs.index_class2],
    cats_dogs.probability(text=df_test.iloc[0]['text'], mode=cats_dogs.index_class2,
                          words=cats_dogs.words_all)
))

### Без учета стоп слов ###
входит в группу "Коты" с вероятностью 0.99
входит в группу "Собаки" с вероятностью 0.01


## Spam/Ham

### Загрузка и подготовка данных

In [12]:
spam_ham = nb.NaiveBayesMultinomial()
spam_ham.stem_func = get_stem
spam_ham.stem_enable = True
spam_ham.stem_lang = 'en'

spam_ham.set_title('Spam', 'Ham')

fraction_test = 0.2

df_raw = pd.read_csv('input/SMSSpamCollection', delimiter='\t', header=None, names=['class1', 'text'])
df_raw['class1'] = np.where(df_raw['class1'] == "spam", 1, 0)

df_test = pd.concat([
    df_raw.loc[df_raw['class1'] == 1].sample(frac=fraction_test, random_state=12345),
    df_raw.loc[df_raw['class1'] == 0].sample(frac=fraction_test, random_state=12345)
])
df = df_raw.drop(df_test.index)

In [13]:
spam_ham.load_stop_words('input/stopwords_en.txt')
count_all, count_spam, count_ham = spam_ham.calculate_counters(df=df, column='class1')

print(f'Данные для расчета: {spam_ham.class_title[spam_ham.index_class1]}: {count_spam}, {spam_ham.class_title[spam_ham.index_class2]}: {count_ham}')
print('Данные для проверки: {0}: {1}, {2}: {3}'.format(
    spam_ham.class_title[spam_ham.index_class1],
    df_test['class1'].loc[df_test['class1'] == 1].count(),
    spam_ham.class_title[spam_ham.index_class2],
    df_test['class1'].loc[df_test['class1'] == 0].count()
))


Данные для расчета: Spam: 598, Ham: 3860
Данные для проверки: Spam: 149, Ham: 965


### Подсчет слов

In [14]:
spam_ham.train(df=df)

print('All - {0}, without stop words - {1}'.format(len(spam_ham.words_all), len(spam_ham.words_without_stop)))

All - 5744, without stop words - 5192


### Проверка на тестовой выборке

#### с учетом стоп слов

In [15]:
### test data process
cnt_good, cnt_bad = spam_ham.test_classifier(
    df_check=df_test,
    words=spam_ham.words_without_stop,
    debug=False
)
spam_ham.print_result(cnt_good, cnt_bad)

Результат:
Тип Spam: попаданий (130), ошибок(7)
Тип Ham: попаданий (958), ошибок(19)
Всего, попаданий: 97.67%



#### без учета стоп слов

In [16]:
### test data process
cnt_good, cnt_bad = spam_ham.test_classifier(
    df_check=df_test,
    words=spam_ham.words_all,
    debug=False
)
spam_ham.print_result(cnt_good, cnt_bad)

Результат:
Тип Spam: попаданий (110), ошибок(0)
Тип Ham: попаданий (965), ошибок(39)
Всего, попаданий: 96.5%



# Категориальный классификатор

## Угон авто

### Загрузка и подготовка данных

In [17]:
stolen_cars = nb.NaiveBayesCategorical()

data_train = [
    ['красный', 'спортивный', 'отечественный', 1],
    ['красный', 'спортивный', 'отечественный', 0],
    ['красный', 'спортивный', 'отечественный', 1],
    ['оранжевый', 'спортивный', 'отечественный', 0],
    ['оранжевый', 'спортивный', 'импортный', 1],
    ['оранжевый', 'внедорожник', 'импортный', 0],
    ['оранжевый', 'внедорожник', 'импортный', 1],
    ['оранжевый', 'внедорожник', 'отечественный', 0],
    ['красный', 'внедорожник', 'импортный', 0],
    ['красный', 'спортивный', 'импортный', 1],
]
columns_train=['цвет', 'тип', 'страна_производства', 'угоняется']

data_check = {'цвет': 'красный', 'тип': 'внедорожник', 'страна_производства': 'отечественный'}

## prepare
df_train = pd.DataFrame(list(data_train), columns=columns_train)
df_train['угоняется'] = np.where(df_train['угоняется'] == 1, 'да', 'нет')
####

print(df_train.head(5))

        цвет         тип страна_производства угоняется
0    красный  спортивный       отечественный        да
1    красный  спортивный       отечественный       нет
2    красный  спортивный       отечественный        да
3  оранжевый  спортивный       отечественный       нет
4  оранжевый  спортивный           импортный        да


In [18]:
stolen_cars.train(df=df_train, class_column='угоняется')
print('Таблица правдоподобия')
print(stolen_cars.get_model())
print()

Таблица правдоподобия
{'да': {'цвет': {'_cnt_': 5, '_data_': {'красный': 3, 'оранжевый': 2}}, 'тип': {'_cnt_': 5, '_data_': {'спортивный': 4, 'внедорожник': 1}}, 'страна_производства': {'_cnt_': 5, '_data_': {'отечественный': 2, 'импортный': 3}}}, 'нет': {'цвет': {'_cnt_': 5, '_data_': {'красный': 2, 'оранжевый': 3}}, 'тип': {'_cnt_': 5, '_data_': {'спортивный': 2, 'внедорожник': 3}}, 'страна_производства': {'_cnt_': 5, '_data_': {'отечественный': 3, 'импортный': 2}}}}



### Проверка

In [19]:
p = stolen_cars.probability(data_check=data_check)
print(f'{p["answer"].capitalize()}? {data_check}')

answer = {'title': '', 'p': 0}
for i in p['probability']:
    if answer['title'] == '' or p['probability'][i] >= answer['p']:
        answer['title'] = i
        answer['p'] = p['probability'][i]
print('{0}: {1}'.format(answer['title'], answer['p']))

print()
print('probability of `{0}` : {1}'.format(p["answer"],p['probability']))

Угоняется? {'цвет': 'красный', 'тип': 'внедорожник', 'страна_производства': 'отечественный'}
нет: 0.144

probability of `угоняется` : {'да': 0.048, 'нет': 0.144}


## Цветы Ирисы

### Загрузка и подготовка данных

In [20]:
iris = nb.NaiveBayesCategorical()


fraction_test = 0.4

df_iris_raw = pd.read_csv('input/Iris.csv', delimiter=',')
df_iris_raw=df_iris_raw.drop(['Id'], axis=1)

df_iris_raw['SepalLengthCm'] = df_iris_raw['SepalLengthCm'].fillna(0)
df_iris_raw['SepalWidthCm'] = df_iris_raw['SepalWidthCm'].fillna(0)
df_iris_raw['PetalLengthCm'] = df_iris_raw['PetalLengthCm'].fillna(0)
df_iris_raw['PetalWidthCm'] = df_iris_raw['PetalWidthCm'].fillna(0)

df_iris_raw.head(5)

df_iris_test = pd.concat([
    df_iris_raw.loc[df_iris_raw['Species'] == 'Iris-setosa'].sample(frac=fraction_test, random_state=12345),
    df_iris_raw.loc[df_iris_raw['Species'] == 'Iris-versicolor'].sample(frac=fraction_test, random_state=12345),
    df_iris_raw.loc[df_iris_raw['Species'] == 'Iris-virginica'].sample(frac=fraction_test, random_state=12345),
])

df_iris_train = df_iris_raw.drop(df_iris_test.index)

print(f'Данные для расчета: {len(df_iris_train)}')
print(f'Данные для проверки: {len(df_iris_test)}')


Данные для расчета: 90
Данные для проверки: 60


In [21]:
iris.train(df=df_iris_train, class_column='Species')
# print('Таблица правдоподобия')
# print(iris.get_model())
# print()

### Проверка на тестовой выборке

In [22]:
test_count = {'ok': 0, 'error': 0}
for index, row in df_iris_test.iterrows():

    reference = row['Species']
    data_check = {
        'SepalLengthCm': row['SepalLengthCm'],
        'SepalWidthCm': row['SepalWidthCm'],
        'PetalLengthCm': row['PetalLengthCm'],
        'PetalWidthCm': row['PetalWidthCm']
    }
    p = iris.probability(data_check=data_check)
    

    answer = {'title': '', 'p': 0}
    for i in p['probability']:
        if answer['title'] == '' or p['probability'][i] >= answer['p']:
            answer['title'] = i
            answer['p'] = p['probability'][i]
    # print(f'{p["answer"].capitalize()}? {data_check}')
    # print('{0}: Эталон: {1}'.format(answer['title'], reference))
    # print()
    if answer['title'] == reference:
        test_count['ok'] += 1
    else:
        test_count['error'] += 1
print('Попаданий: {0}, ошибок {1}'.format(test_count['ok'],test_count['error']))
print('Точность {0}%'.format(round(test_count['ok']*100/(test_count['ok']+test_count['error']))))

Попаданий: 54, ошибок 6
Точность 90%
